In [1]:
import os 
import pickle
import torch

dataset = 'Cora'
path = f'/home/jrm28/graph_transformer/Exphormer/activations/{dataset}/Random-d/'
epochs = len(list(filter(lambda x: "Q" in x, os.listdir(path))))

Q, K, V, scores, edge_indexes = [], [], [], [], []

print(f"reading {epochs} epochs of activations from {path}")
for epoch in range(100):
    for layer in ('Q', 'K', 'V', 'scores', 'edge_indexes'):
        file = f'{epoch}_{layer}_activation.pkl'
        outp = torch.load(os.path.join(path, file), map_location=torch.device('cpu'))
        
        if layer == 'Q':
            Q.append(outp)
        elif layer == 'K':
            K.append(outp)
        elif layer == 'V':
            V.append(outp)
        elif layer == 'edge_indexes':
            edge_indexes.append(outp)
        else:
            scores.append(outp)


reading 100 epochs of activations from /home/jrm28/graph_transformer/Exphormer/activations/Cora/Random-d/


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [2]:
import torch
import math

In [3]:
edge_indexes[0][0][0]

tensor([   0,    0,    0,  ...,  542, 1931, 2344], device='cuda:0')

In [4]:
scores[0][0][:, 0].shape

torch.Size([40068, 1])

In [5]:
num_nodes = Q[0][0].shape[0]
attention_matrices = torch.zeros(num_nodes, num_nodes)
attention_matrices[edge_indexes[0][0][0], edge_indexes[0][0][1]] = scores[0][0][:, 0].squeeze().cpu().detach()


In [6]:
len(scores)

100

In [7]:
def create_attention_matrices(scores, edge_indexes, num_nodes):
    attention_matrices = torch.zeros(len(scores), num_nodes, num_nodes)
    for i, (score, edge_index) in enumerate(zip(scores, edge_indexes)):
        print(f"Epoch {i} - score shape", score[0][:, 0].shape)
        attention_matrices[i, edge_index[0][0], edge_index[0][1]] = score[0][:, 0].squeeze().cpu().detach()
    return attention_matrices

In [10]:
len(scores[::10])

10

In [ ]:
import matplotlib.backends.backend_pdf
import matplotlib.pyplot as plt


tensors = create_attention_matrices(scores[::10], edge_indexes, Q[0][0].shape[0])

In [ ]:
# File path for the PDF
pdf_path = './tensors_heatmap.pdf'

# Create a PDF file with each page containing one heatmap
# with matplotlib.backends.backend_pdf.PdfPages(pdf_path) as pdf:
for timestamp, tensor in tensors:
    fig, ax = plt.subplots(figsize=(8, 6))
    cax = ax.matshow(tensor, cmap='viridis')
    fig.colorbar(cax)
    ax.set_title(f'Heatmap at {timestamp}')
    pdf.savefig(fig)  # Save the current figure into the PDF
    plt.close(fig)  # Close the figure to free memory


In [ ]:
# File path for the PDF
pdf_path = './tensors_heatmap.pdf'

# Create a PDF file with each page containing one heatmap
with matplotlib.backends.backend_pdf.PdfPages(pdf_path) as pdf:
    for timestamp, tensor in tensors:
        fig, ax = plt.subplots(figsize=(8, 6))
        cax = ax.matshow(tensor, cmap='viridis')
        fig.colorbar(cax)
        ax.set_title(f'Heatmap at {timestamp}')
        pdf.savefig(fig)  # Save the current figure into the PDF
        plt.close(fig)  # Close the figure to free memory

pdf_path

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


tensors = [(f"Epoch {i}", attention_fn(Q[i][0], K[i][0]).cpu().numpy()) for i in range(epochs)]

# Function to plot a heatmap for a single tensor
def plot_tensor_heatmap(ax, tensor, timestamp, cmap='viridis'):
    ax.clear()  # Clear the current axes
    heatmap = ax.imshow(tensor, cmap=cmap) # Plot the heatmap
    ax.set_title(f"Heatmap at {timestamp}") # Set the title with the timestamp
    return heatmap

# Setup the figure and axis for animation
fig, ax = plt.subplots(figsize=(8, 6))

# Animation function to update the plot with a new tensor
def animate(i):
    print(i)
    timestamp, tensor = tensors[i]
    return plot_tensor_heatmap(ax, tensor, timestamp)

# Create the animation
ani = animation.FuncAnimation(fig, animate, frames=epochs, interval=500, blit=False)

plt.close()  # Avoid displaying the static plot

# Save the animation to a GIF file
animation_path = '/tensor_heatmaps.gif'
ani.save(animation_path)

animation_path

MovieWriter ffmpeg unavailable; using Pillow instead.


0


IndexError: list index out of range